In [1]:
from torch import nn
import math
import torch
# import numpy as np
import torch.nn.functional as F


class MeanVarianceLoss(nn.Module):

    def __init__(self, lambda_1, lambda_2, start_age, end_age):
        super().__init__()
        self.lambda_1 = lambda_1
        self.lambda_2 = lambda_2
        self.start_age = start_age
        self.end_age = end_age

    def forward(self, input, target):

        N = input.size()[0]
        target = target.type(torch.FloatTensor).cuda()
        m = nn.Softmax(dim=1)
        p = m(input)
        # print('input', input.size())
        # print('target', target.size())
        # print('p', p.size())
        # mean loss
        a = torch.arange(self.start_age, self.end_age + 1, dtype=torch.float32).cuda()
        mean = torch.squeeze((p * a).sum(1, keepdim=True), dim=1)
        mse = (mean - target)**2
        mean_loss = mse.mean() / 2.0
        # print('mean', mean.size())
        # print('mse', mse.size())
        # print('mean_loss', mean_loss)

        # variance loss
        b = (a[None, :] - mean[:, None])**2
        variance_loss = (p * b).sum(1, keepdim=True).mean()
        # print(variance_loss)
        
        # return self.lambda_1 * mean_loss, torch.tensor(0.)
        return self.lambda_1 * mean_loss, self.lambda_2 * variance_loss
    
    
class MeanVarianceLoss2(nn.Module):

    def __init__(self, lambda_1, lambda_2, start_age, end_age):
        super().__init__()
        self.lambda_1 = lambda_1
        self.lambda_2 = lambda_2
        self.start_age = start_age
        self.end_age = end_age

    def forward(self, input, target):

        N = input.size()[0]
        target = target.type(torch.FloatTensor).cuda()
        m = nn.Softmax(dim=1)
        p = m(input)
        # print('input', input.size())
        # print('target', target.size())
        # print('p', p.size())
        # mean loss
        a = torch.arange(self.start_age, self.end_age + 1, dtype=torch.float32).cuda()
        mean = torch.squeeze((p * a).sum(1, keepdim=True), dim=1)
        #variance = 
        b = (a[None, :] - mean[:, None])**2 # line 78
        
        #variance = (p * b).sum(1, keepdim=True)        
        variance = (p * b).sum(1)
        
        mse = ((mean - target)/variance)**2
        mean_loss = mse.mean() / 2.0
        # print('mean', mean.size())
        # print('mse', mse.size())
        # print('mean_loss', mean_loss)

        # variance loss
        #b = (a[None, :] - mean[:, None])**2
        variance_loss = variance.mean()
        # print(variance_loss)
        
        # return self.lambda_1 * mean_loss, torch.tensor(0.)
        return self.lambda_1 * mean_loss, self.lambda_2 * variance_loss


In [2]:
lossfun = MeanVarianceLoss2(0.2, 0.005, 0, 10)

In [3]:
import torch

In [8]:
import numpy as np
device = 'cpu'

In [9]:
class FNN(nn.Module):
    def __init__(self, nclass=None, ys = None):
        super().__init__()
        nnode = 5
        #nnode = 30
        self.nn = nn.Sequential(
            nn.Linear(1,nnode),  # num param = 11
            #nn.LeakyReLU(10), # num param = 11
  
            #nn.Linear(10,10), # num param = 110
            #nn.LeakyReLU(10), # num param = 10
            
            nn.Linear(nnode, nclass), # num param = (10+1)*nclass  
            # the number of classes
        )
        self.softmax = nn.Softmax(dim=1)
        if nclass is not None:
            self.nclass = nclass
        else:
            raise ValueError('nclass is required')
            
        if ys is not None:
            self.ys = ys
        else:
            raise ValueError('ys is required')
            
        if nclass != len(ys):
            raise ValueError('nclass should equal to len(ys)')

    def forward(self, x):
        x = self.nn(x)
        #x = self.softmax(x)
        return x
    

model = FNN(nclass = 11, ys = np.arange(11))
model.to(device)

FNN(
  (nn): Sequential(
    (0): Linear(in_features=1, out_features=5, bias=True)
    (1): Linear(in_features=5, out_features=11, bias=True)
  )
  (softmax): Softmax(dim=1)
)

In [10]:
ypred = torch.tensor([1,2,3,4,5,6,7,8,9,10])

In [14]:
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='uint8')[y]

z = to_categorical(ypred, 11)
z

array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=uint8)

In [11]:
ytrue = torch.tensor([0,1,2,4,5,3,2,5,6,7])

In [ ]:
mean = torch.squeeze((p * a).sum(1, keepdim=True), dim=1)

In [31]:
torch.squeeze((torch.tensor(z)*torch.tensor(z)).sum(1, keepdim=True), dim=1)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [16]:
z_ = torch.tensor(z, dtype=torch.float32)

In [24]:
lossfun(z_.to('cuda'), ytrue.to('cuda'))

(tensor(0.0062, device='cuda:0'), tensor(0.0482, device='cuda:0'))

In [22]:
z_.device

device(type='cpu')

In [23]:
ytrue.device

device(type='cpu')

In [20]:
z_.to('cuda')

tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], device='cuda:0')

In [21]:
ytrue.to('cuda')

tensor([0, 1, 2, 4, 5, 3, 2, 5, 6, 7], device='cuda:0')